In [1]:
import pandas as pd
import plotly.express as px
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet, HuberRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import root_mean_squared_error, r2_score

### Carregar os dados

In [2]:
# Carregar os dados ja tratados
df_costs = pd.read_csv('.\datasets\healthcost_cleaned.csv')

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Carolina\AppData\Local\Temp\ipykernel_11804\2116839817.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_costs = pd.read_csv('.\datasets\healthcost_cleaned.csv')


In [3]:
df_costs.head(10)

,age,sex,bmi,children,smoker,region,medical charges
0,19,female,27.900,0,1,southwest,16884.92400
1,18,male,33.770,1,0,southeast,1725.55230
2,28,male,33.000,3,0,southeast,4449.46200
3,33,male,22.705,0,0,northwest,21984.47061
4,32,male,28.880,0,0,northwest,3866.85520
5,31,female,25.740,0,0,southeast,3756.62160
6,46,female,33.440,1,0,southeast,8240.58960
7,37,female,27.740,3,0,northwest,7281.50560
8,37,male,29.830,2,0,northeast,6406.41070
9,60,female,25.840,0,0,northwest,28923.13692


In [5]:
df_costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              1338 non-null   int64  
 1   sex              1338 non-null   object 
 2   bmi              1338 non-null   float64
 3   children         1338 non-null   int64  
 4   smoker           1338 non-null   int64  
 5   region           1338 non-null   object 
 6   medical charges  1338 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 73.3+ KB


# Preparação dos Dados

In [6]:
# Preparar os dados para o modelo
X = df_costs.drop(columns=['medical charges'])
y = df_costs['medical charges']

In [8]:
# Importar preprocessor
import joblib
preprocessor = joblib.load('.\preprocessor_dataset_healthcosts.pkl')

<>:3: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Carolina\AppData\Local\Temp\ipykernel_11804\3149317600.py:3: SyntaxWarning: invalid escape sequence '\p'
  preprocessor = joblib.load('.\preprocessor_dataset_healthcosts.pkl')


In [10]:
# Dividir os dados entre treino e teste
X_train, X_test, y_train_, y_test = train_test_split(X, y, test_size=0.2, random_state=51)

In [11]:
# Aplicar preprocessor em treinamento e test
# Treinamento = fit e transform
# Teste = tranform, considerando o treinamento que foi feito anteriormente
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [13]:
# Mostrar as dimensões dos conjuntos
print(f'Treinamento: {X_train.shape}\nTeste: {X_test.shape}')

Treinamento: (1070, 10)
Teste: (268, 10)


### Treinamento do modelo Stacking

In [15]:
# Criar o modelo de StackingRegressor
# Algoritmo base
lr_model = LinearRegression()
elastic_model = ElasticNet(alpha=1.0, l1_ratio=0., random_state=51)
tree_model = DecisionTreeRegressor(random_state=51)

# Meta-Modelo ou Meta-Learner
huber_model = HuberRegressor()

# Stacking Model
stacking_model = StackingRegressor(
    estimators=[
        ('linear regression', lr_model),
        ('elastc', elastic_model),
        ('decision tree', tree_model)
    ],
    final_estimator=huber_model,
    # passthrough=False (padrão) -> Usa apenas as predições dos estimadores
    # passthrough=True -> Usa as predições dos estimadores mais o conjunto de treinamento
    passthrough=False
)

In [16]:
# Treinar o modelo
stacking_model.fit(X_train, y_train_)

c:\Users\Carolina\.virtualenvs\ROCKETSEAT-znLm4meT\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.685e+10, tolerance: 1.494e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
c:\Users\Carolina\.virtualenvs\ROCKETSEAT-znLm4meT\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.716e+10, tolerance: 1.182e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted u

StackingRegressor(estimators=[('linear regression', LinearRegression()),
                              ('elastc',
                               ElasticNet(l1_ratio=0.0, random_state=51)),
                              ('decision tree',
                               DecisionTreeRegressor(random_state=51))],
                  final_estimator=HuberRegressor())

### Análise de Resultados

In [17]:
# Fazer as predições com base no modelo treinado
y_pred = stacking_model.predict(X_test)

In [18]:
# Mostrar as predições 
y_pred

array([ 7714.86109043, 43360.22249709, 20520.07321063, 23282.31048848,
       38524.54802489,  9702.53155394,  7856.75117667, 12182.28964544,
        5549.70121501,  9424.0019919 ,  8709.92984442, 11497.1835868 ,
        7318.89022491,  2455.59641388,  4553.1229559 , 12491.97590241,
        3093.51741636,  6938.6146299 , 18351.64779524, 20120.83934191,
        4900.60285255,  6822.55851137, 53523.70887869, 10875.92272134,
        5626.51820581, 15005.69957163, 11475.74927066,  1576.11144315,
       30816.46263819, 19318.78680964,  1449.57517419, 23009.5273374 ,
        2480.40917299,  2806.56484278,  7053.36211572, 24963.03987375,
        7369.54834066,  1419.98411948, 11647.41742458,  7257.69078566,
       11210.99227355,  1234.02204752,  3677.06240401,  1463.21764929,
       12228.48584677, 12743.41467641, 11567.75084243, 40214.34835611,
        8301.09264198, 12548.90908085,  4748.57028957, 37477.73663171,
        8823.13217116, 46787.99173063, 18813.67989419, 33236.61739427,
      

In [19]:
# Avaliar Métricas do modelo
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [20]:
# Mostrar o Error e R2 do modelo
print(f'RMSE: {rmse:.2f}\nR2 Score: {r2:.2f}')

RMSE: 6637.38
R2 Score: 0.75


In [25]:
# Calcular a imporância considerando os modelos do stacking regressor

importances = []

for estimator in stacking_model.estimators:
    if hasattr(estimator, 'coef_'):
        # Uso em modelos Lineares
        importances.append(np.abs(estimator.coef_))
    elif hasattr(estimator, 'feaeture_importances_'):
        # Uso em modelos de árvore
        importances.append(estimator.feature_importances_)
    else:
        print(f'Não foi possivel carregar a importância das variáveis do modeo {estimator}')

Não foi possivel carregar a importância das variáveis do modeo ('linear regression', LinearRegression())
Não foi possivel carregar a importância das variáveis do modeo ('elastc', ElasticNet(l1_ratio=0.0, random_state=51))
Não foi possivel carregar a importância das variáveis do modeo ('decision tree', DecisionTreeRegressor(random_state=51))


In [26]:
# Calculando a média das importâncias
importancia_media = np.mean(importances, axis=0)

# Obter os nomes da features
features_name = preprocessor.get_feature_names_out()

c:\Users\Carolina\.virtualenvs\ROCKETSEAT-znLm4meT\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Carolina\.virtualenvs\ROCKETSEAT-znLm4meT\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [27]:
# Normalizar as importâncias
feaeture_importance = importancia_media / np.sum(importancia_media)

In [28]:
# Criar um dataframe com as importâncias e os nomes das features
importance_df = pd.DataFrame({'feature': features_name, 'importance': feaeture_importance})

In [29]:
# Ordernar o DataFrame pela importância
importance_df = importance_df.sort_values(by='importance', ascending=True)

In [30]:
# Criar o gráfico de barras para apresentar a importância das features
fig = px.bar(importance_df,
             x='importance',
             y='feature',
             title='Importância das Features - Stacking Regressor',
             orientation='h')
fig.show()